# Simple Script for Creating Hopfield Networks

### Identification of Pattern Completion Neurons in Neuronal Ensembles using Probabilistic Graphical Models

### Luis Carrillo-Reid, Shuting Han, Darik O'Neil, Ekaterina Taralova, Tony Jebara, Rafael Yuste

#### Methods were based on the descriptions provided within the textbook Neuronal Dynamics by Wulfram Gerstner, Werner M. Kistler, Richard Naud, and Liam Paninski (Cambridge Press, 2014).

In [1]:
#IMPORT CLASSES & FUNCTIONS
from HopCRF.HopNet import HopNet
from HopCRF.HopPat import HopPat
from HopCRF.tools import tools
import math

In [2]:
#IMPORT NUMPY & H5PY FOR EXPORTING DATA
import numpy as np
import h5py

In [3]:
#set params
meanActivity = 7.5 #percent
subgraphSize = 81 #must be a perfect square
numSubgraphs = 10
ensSubgraph = 10
buffer_activity = tools.calculate_buffer(meanActivity, subgraphSize, numSubgraphs)


In [4]:
#solve
HopfieldNet = []
Noise = []
StateTemp = []
StatesForCRFs = []

for i in range(numSubgraphs):
    hopfieldNet = HopNet(subgraphSize)
    hopPatterns = HopPat(int(math.sqrt(subgraphSize)))
    pattern_list = []
    pattern_list.extend(hopPatterns.create_random_pattern_list(nr_patterns=10,on_probability=0.5))
    hopfieldNet.store_pattern(pattern_list)
    noise = []
    stateTemp = []
    statesForCRFs = []
    STATES_FOR_CRFS = []
    for i in range(ensSubgraph):
        noisey = tools.get_noisy_copy(pattern_list[i],noise_level=0.2)
        noise.append(noisey)
        hopfieldNet.set_state_from_pattern(noisey)
        STATES_TEMP = hopfieldNet.run(nr_steps=5)
        stateTemp.append(STATES_TEMP)
        STATES_FOR_CRFS = hopPatterns.reshape_patterns(STATES_TEMP)
        statesForCRFs.append(STATES_FOR_CRFS)
    HopfieldNet.append(hopfieldNet)
    Noise.append(noise)
    StateTemp.append(stateTemp)
    StatesForCRFs.append(statesForCRFs)

In [5]:
#Save for importing to MATLAB
f = h5py.File("SIM_MODEL_CRF2.hdf5", "w")
f.create_dataset("StatesForCRFs",data=StatesForCRFs)
f.create_dataset("Noise",data=Noise)
f.create_dataset("StateTemp",data=StateTemp)
for i in range(0,(len(HopfieldNet)-1)):
    weightnum = str(i)
    f.create_dataset(weightnum, data=HopfieldNet[i].weights)
f.close()